# GO2 Learning to stand via genetic algo 

## Setup

In [ ]:
import numpy as np
import mujoco
import mujoco.viewer
import mediapy as media
from robot_descriptions import go2_mj_description

### Import Model

Let's take a look at the model xml

In [ ]:
with open(go2_mj_description.MJCF_PATH, 'r') as file:
    model_xml = file.read()
print(model_xml)

print(go2_mj_description.MJCF_PATH)

In [ ]:
model = mujoco.MjModel.from_xml_path(go2_mj_description.PACKAGE_PATH + "/scene.xml")
data = mujoco.MjData(model)

renderer = mujoco.Renderer(model)

## Experimentation

First up, lets render the scene

In [ ]:
duration = 3.8  # (seconds)
framerate = 60  # (Hz)

frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

Plan is to create a neural network to control the robot. 
What are the inputs?
Robot needs to be aware of state (joint angles, joint velocities)

See https://mujoco.readthedocs.io/en/stable/python.html#named-access

The model has names associated with joints (e.g. `<joint name="FR_hip_joint" class="abduction"/>`).  
Named joints can be accessed on the `mujoco.MjData` object via `data.jnt("FR_hip_joint")`.

In [ ]:
joint_names = [mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, i) for i in range(1, model.njnt)]
print("Joint Names:", joint_names)
for joint_name in joint_names:
    print(f"{joint_name}: qpos: {data.jnt(joint_name).qpos}, qvel: {data.jnt(joint_name).qvel}")

The network should output the torques to be applied at each joint.
Let's try to control a joint using torque control.

In [ ]:
frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    data.jnt("FL_thigh_joint").qvel = 2 
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)